# Brain Tumour Detection using MRI

## Image Processing: Digital Assignment 1
## Done by Ritika Kayal, 18BCE2518


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

from skimage.io import imread
from skimage.feature import greycomatrix, greycoprops
from sklearn import preprocessing
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Reading the Data

In [ ]:
location = "../input/raw-images/brain_tumor_dataset/yes/Y3.jpg";
img = cv2.imread(location,1)
img.shape

In [ ]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
original = img
original.shape
row, columns, channels = original.shape

In [ ]:
plt.imshow(original)
plt.title("Original Image")
plt.show()

## Image Enhancement and Restoration Techniques

## Color Conversion

In [ ]:
gray_img = cv2.cvtColor( original, cv2.COLOR_BGR2GRAY ) 
plt.imshow(gray_img,cmap='gray')
plt.title("Grayscale Image")
plt.show()

## Salt & Pepper Noise Removal

In [ ]:
median_output = cv2.medianBlur(gray_img, 5)
plt.imshow(median_output,cmap='gray')
plt.show()

## Edge Detection using : 3*3 Sobel Filters


In [ ]:
Gx= np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
Gy = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

sobelx = cv2.Sobel(median_output,cv2.CV_8U,1,0,ksize=3)
sobely = cv2.Sobel(median_output,cv2.CV_8U,0,1,ksize=3)

sobel_output = sobelx + sobely + gray_img
plt.imshow(sobel_output,cmap='gray')
plt.show()

## Thresholding

In [ ]:
threshold = 127
maxValue = 255

th, threshold_filtered = cv2.threshold(sobel_output, threshold, maxValue, cv2.THRESH_BINARY)
plt.imshow(threshold_filtered,cmap='gray')
plt.axis('off')
plt.show()

## Morphological Processing 
### To remove any small white noises 

In [ ]:
kernel = np.ones((3,3),np.uint8)
morph = cv2.morphologyEx(threshold_filtered,cv2.MORPH_OPEN,kernel, iterations = 2)

plt.imshow(morph,cmap='gray')
plt.axis('off')
plt.show()

## Black region removal
### To remove sure background area

In [ ]:
removed_bg = cv2.dilate(morph,kernel,iterations=3)

plt.imshow(removed_bg,cmap='gray')
plt.axis('off')
plt.show()

##  White region identification
### To identify sure foreground area

In [ ]:
dist = cv2.distanceTransform(morph,cv2.DIST_L2,5)
ret, fg = cv2.threshold(dist,0.7*dist.max(),255,0)

plt.imshow(fg,cmap='gray')
plt.axis('off')
plt.show()

## Image Segmentation
### Identifying regions where we don't know whether foreground and background using Watershed algorithm

In [ ]:
fg = np.uint8(fg)
unknown = cv2.subtract(removed_bg,fg)

plt.imshow(unknown,cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
contours, _ = cv2.findContours(fg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
marker = np.zeros((gray_img.shape[0], gray_img.shape[1]),dtype = np.int32)
marker = np.int32(fg) + np.int32(removed_bg)

# Marker Labelling
for id in range(len(contours)):
    cv2.drawContours(marker,contours,id,id+2, -1)

marker = marker + 1
marker[unknown==255] = 0
copy_img = original.copy()
cv2.watershed(copy_img, marker)
imgplt = plt.imshow(marker)
plt.colorbar()
plt.show()

In [ ]:
copy_img[marker==-1]=(0,0,255)
plt.imshow(copy_img,cmap='gray')
plt.axis('off')
plt.title('Detected Tumor Image')
plt.show()

## Feature Extraction for Further Classification

In [ ]:
input_data_dir = '../input/input-data/MRI_Dataset/Train'

## Feature Extraction for Further Classification

In [ ]:
labeling_class = {
  "BENIGN": 0,
  "MALIGNANT": 1,
  "NORMAL": 2
}

no_cols = 11
cols =np.asarray(['Contrast','Energy','Homogeneity','Correlation',
                  'Dissimilarity','ASM','Area','Perimeter','Epsilon','IsConvex','Class'])

no_samples = 0
labels = os.listdir(input_data_dir)
for label in labels:
    no_samples += len(os.listdir(os.path.join(input_data_dir,label)))
print('Number of samples : ',no_samples)

In [ ]:
def extract_feature():
    features = np.ndarray((no_samples, no_cols))
    
    labels = os.listdir(input_data_dir)
    print(labels)
    i = 0
    for label in labels:
        images = os.listdir(os.path.join(input_data_dir, label))
        total = len(images)
        print(total)
        for image_name in images:
            img = imread(os.path.join(input_data_dir, label, image_name))
            
            ret,thresh = cv2.threshold(img,127,255,0)
            contours,hierarchy = cv2.findContours(thresh, 1, 2)
            c = contours[0]

            area = cv2.contourArea(c)
            perimeter = cv2.arcLength(c,True)
            epsilon = 0.1*cv2.arcLength(c,True)
            k = cv2.isContourConvex(c)
            S = preprocessing.MinMaxScaler((0,11)).fit_transform(img).astype(int)
            g = greycomatrix(S, distances=[1], angles=[0], levels=256, symmetric=False, normed=False)

            contrast = greycoprops(g, 'contrast')
            energy = greycoprops(g,'energy')
            homogeneity = greycoprops(g, 'homogeneity')
            correlation = greycoprops(g, 'correlation')
            dissimilarity = greycoprops(g, 'dissimilarity')
            ASM = greycoprops(g, 'ASM')
            
            f_arr =  np.asarray([contrast[0][0],energy[0][0],homogeneity[0][0],correlation[0][0],dissimilarity[0][0],
                                 ASM[0][0],area,perimeter,epsilon,k, labeling_class[label]], dtype='object')
            features[i] = f_arr
            i += 1
    df = pd.DataFrame(features, columns=cols)
    df['Class'] = np.int64(df['Class'])
    df.to_csv("data.csv",index= False)

extract_feature()

In [ ]:
data_as_frame = pd.read_csv('data.csv')
data_as_frame.head()